In [2]:
from datetime import datetime

In [3]:
def extract_dates_from_section(section):
    # Recherche de la balise h3 pour déterminer le type de réunion
    meeting_type = section.find('h3').text.strip()

    # Recherche des balises <p> pour extraire les dates
    date_paragraphs = section.find_all('p')

    # Extraction des dates
    dates = [date.text.strip() for date in date_paragraphs]

    return meeting_type, dates



In [19]:
import requests
from bs4 import BeautifulSoup
import re
year = [2007,2009,2011, 2013] # je voulais faire une boucle pour changer la date dans l'url mais il y a un problème avec le "pr090508" qui change à chaque année donc compliqué de récupérer 
url = "https://www.ecb.europa.eu/press/pr/date/2009/html/pr090508.en.html"

# Envoyer une requête HTTP pour obtenir le contenu de la page
response = requests.get(url)

# Vérifier le statut de la requête
if response.status_code == 200:
    dates = {"Governing Council": [], "General Council": [], "Press conferences": [], "All Dates": []}

    # Analyser le contenu HTML avec BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Rechercher la balise <div class="section" lang="en">
    section_divs = soup.find_all('div', class_='section', lang='en')
    for section_div in section_divs:
        # Rechercher la balise <h3> pour déterminer le type de réunion
        meeting_type = section_div.find('h3').text.strip()

        # Utiliser une fonction pour convertir la phrase en une clé plus générique
        def convert_meeting_type(meeting_type):
            if "Governing Council" in meeting_type:
                return "Governing Council"
            elif "General Council" in meeting_type:
                return "General Council"
            elif "Press conferences" in meeting_type:
                return "Press conferences"
            else:
                return "Autre" 

        # Utiliser la fonction pour obtenir la clé
        key = convert_meeting_type(meeting_type)

        # Rechercher les balises <p> pour extraire les dates
        dates_div = section_div.find_all("p")
        for date in dates_div:

            date_match_two_days = re.search(r'(\d+/\d+)\s(\w+\s\d+)', date.text.strip() )
            
            if date_match_two_days:
                jours, mois_annee = date_match_two_days.groups()
                jours_list = re.split(r'[-/]', jours)

                # Combinaison des jours avec le mois et l'année pour avoir les jours de conseil distincts
                dates_list = [f"{jour} {mois_annee}" for jour in jours_list]

                # Suppression des noms de villes/pays
                dates_without_city = [re.search(r'\d+\s\w+\s\d+', d).group() if re.search(r'\d+\s\w+\s\d+', d) else None for d in dates_list]

                # Ajout des dates à la liste
                dates[key].extend(dates_without_city)
                dates["All Dates"].extend(dates_without_city)

            else :
                date_match_without_city = re.search(r'\d+\s\w+\s\d+', date.text.strip()) # Pour enlever les noms des villes/pays
                date_text = date_match_without_city.group() if date_match_without_city else None

                dates[key].append(date_text)
                dates["All Dates"].append(date_text)

    # Afficher les résultats
    for meeting_type, meeting_dates in dates.items():
        print(f"Dates pour {meeting_type} :", meeting_dates)

else:
    print(f"Échec de la requête. Code d'état : {response.status_code}")


Dates pour Governing Council : ['14 January 2010', '4 February 2010', '18 February 2010', '4 March 2010', '17 March 2010', '18 March 2010', '8 April 2010', '22 April 2010', '6 May 2010', '20 May 2010', '10 June 2010', '23 June 2010', '24 June 2010', '8 July 2010', '22 July 2010', '5 August 2010', '2 September 2010', '15 September 2010', '16 September 2010', '7 October 2010', '21 October 2010', '4 November 2010', '18 November 2010', '2 December 2010', '15 December 2010', '16 December 2010', '13 January 2011', '3 February 2011', '17 February 2011', '3 March 2011', '16 March 2011', '17 March 2011', '7 April 2011', '20 April 2011', '5 May 2011', '19 May 2011', '9 June 2011', '21 June 2011', '22 June 2011', '7 July 2011', '21 July 2011', '4 August 2011', '8 September 2011', '21 September 2011', '22 September 2011', '6 October 2011', '20 October 2011', '3 November 2011', '17 November 2011', '8 December 2011', '21 December 2011', '22 December 2011', '12 January 2012']
Dates pour General Counc